In [1]:
# 파이썬 시각화 패키지 불러오기
import matplotlib.pyplot as plt
%matplotlib inline
# 사용자 운영체제 확인
import platform
platform.system()
# 운영체제별 한글 폰트 설정
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'

In [3]:
import re
import numpy as np
import pandas as pd
import json
import requests
import branca
import folium
from folium import plugins
folium.__version__

'0.11.0'

In [3]:
df=pd.read_csv("jeju_list.csv")

In [5]:
df.fillna('-',inplace=True)

In [6]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   가게명         256 non-null    object 
 1   대분류         256 non-null    object 
 2   중분류         256 non-null    object 
 3   소분류         256 non-null    object 
 4   mean_score  256 non-null    float64
 5   주소          256 non-null    object 
 6   Latitude    256 non-null    float64
 7   Longitude   256 non-null    float64
dtypes: float64(3), object(5)
memory usage: 16.1+ KB


,가게명,대분류,중분류,소분류,mean_score,주소,Latitude,Longitude
0,상춘재,식당,한식,"돌문어 비빔밥, 게장",4.1,제주특별자치도 제주시 조천읍 선진길 26,33.458659,126.704844
1,남양수산,식당,일식,"회, 고등어",4.6,제주특별자치도 서귀포시 성산읍 고성동서로56번길 11,33.450135,126.914224
2,선흘곶,식당,한식,정통 한정식,4.5,제주특별자치도 제주시 조천읍 동백로 102,33.508187,126.718049
3,스시호시카이,식당,일식,"회, 초밥, 오마카세",4.6,제주특별자치도 제주시 오남로 90,33.481286,126.516769
4,미영이네식당,식당,일식,"회, 고등어, 방어, 참치",4.5,제주특별자치도 서귀포시 대정읍 하모항구로 42,33.217651,126.249904


In [58]:
jeju_df=pd.read_excel("jejuSIG.xlsx")
jeju_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  467 non-null    int64  
 1   시군구         467 non-null    object 
 2   전용/연면적      467 non-null    float64
 3   거래금액        467 non-null    int64  
 4   평당 거래금액     467 non-null    int64  
 5   코드          467 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 22.0+ KB


In [59]:
jeju_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [27]:
emd_code_extract=list(EMD['코드'])

In [54]:
jeju_df

,시군구,전용/연면적,거래금액,평당 거래금액,코드,읍면동,리
0,제주특별자치도 서귀포시 강정동,372.88,179000,480,50130116,강정동,None
1,제주특별자치도 서귀포시 강정동,163.24,65000,398,50130116,강정동,None
2,제주특별자치도 서귀포시 남원읍 위미리,395.01,210000,531,50130253,남원읍,위미리
3,제주특별자치도 서귀포시 남원읍 위미리,40.80,13500,330,50130253,남원읍,위미리
4,제주특별자치도 서귀포시 남원읍 위미리,40.80,15500,379,50130253,남원읍,위미리
...,...,...,...,...,...,...,...
462,제주특별자치도 제주시 회천동,4.38,1068,243,50110133,회천동,None
463,제주특별자치도 제주시 회천동,4.38,1263,288,50110133,회천동,None
464,제주특별자치도 제주시 회천동,37.93,22000,580,50110133,회천동,None
465,제주특별자치도 제주시 회천동,4.38,2700,616,50110133,회천동,None


In [67]:
jeju_df['읍면동']=None
jeju_df['리']=None
#읍면동, 리 따로 추출해서 합치기: 목적: 시세가 높은 곳 별로 숫자 아이콘으로 순위 매기기 및 마우스 올리면 가격과 동읍면리 나타내기
for i,j in enumerate (jeju_df['시군구']):
    jeju_df['읍면동'][i]=j.split(" ")[2]
    if len(j.split(" "))<4:
        jeju_df['리'][i]=None
    else:
        jeju_df['리'][i]=j.split(" ")[3]

<ipython-input-67-d3cb67802522>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jeju_df['읍면동'][i]=j.split(" ")[2]
<ipython-input-67-d3cb67802522>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jeju_df['리'][i]=None
<ipython-input-67-d3cb67802522>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jeju_df['리'][i]=j.split(" ")[3]


In [68]:
jeju_emdli=jeju_df[['코드','읍면동','리']].drop_duplicates("코드")

In [70]:
jeju_emdli.head()

,코드,읍면동,리
0,5013011600,강정동,None
2,5013025327,남원읍,위미리
18,5013025029,대정읍,보성리
24,5013025021,대정읍,상모리
25,5013025023,대정읍,안성리


In [91]:
jeju_emd_only=jeju_emdli[['코드','읍면동']]
jeju_li_only=jeju_emdli[['코드','리']]

emd merge -> //100
li merge

In [142]:
EMD_with_duplicates=pd.merge(EMD, jeju_emd_only, on='코드')

In [151]:
EMD_with_duplicates

,코드,거래금액,읍면동
0,50110101,136302.0,일도일동
1,50110102,32269.0,일도이동
2,50110103,59242.0,이도일동
3,50110104,71972.0,이도이동
4,50110105,25025.0,삼도일동
...,...,...,...
84,50130320,29000.0,표선면
85,50130320,16000.0,표선면
86,50130320,25000.0,표선면
87,50130320,35000.0,표선면


# 제주 "읍면동" 평균 거래금액

jeju_EMD=jeju_df[jeju_df['코드']%100==0]
jeju_EMD

##### 이렇게 하면 "리" 가 포함된 "읍면동"도 제거되서 "리"만 따로 자세히 뽑을때 이 기법을 사용함
##### 리를 포함한 읍면동을 평균으로 뽑으려면 다른방법 사용

In [145]:
EMD_with_duplicates['코드']=EMD_with_duplicates['코드']//100

In [146]:
EMD_mean_price=EMD_with_duplicates.groupby(['코드'])['거래금액'].mean().round(0)

In [157]:
len(EMD_mean_price)

52

In [175]:
EMD_mean_price2=pd.DataFrame(EMD_mean_price)

In [177]:
EMD_mean_price2.reset_index(inplace=True)

In [178]:
EMD_mean_price2

,코드,거래금액
0,50110101,136302.0
1,50110102,32269.0
2,50110103,59242.0
3,50110104,71972.0
4,50110105,25025.0
5,50110106,30978.0
6,50110107,29465.0
7,50110108,27860.0
8,50110109,18000.0
9,50110110,34333.0


In [160]:
EMD_with_duplicates_name=EMD_with_duplicates.copy()

In [163]:
EMD_with_duplicates_name2=EMD_with_duplicates_name[['코드','읍면동']].drop_duplicates('코드')

In [165]:
len(EMD_with_duplicates_name2)

52

In [166]:
EMD_with_duplicates_name2.reset_index(inplace=True)

In [168]:
EMD_with_duplicates_name2.drop('index', axis=1, inplace=True)

In [169]:
EMD_with_duplicates_name2

,코드,읍면동
0,50110101,일도일동
1,50110102,일도이동
2,50110103,이도일동
3,50110104,이도이동
4,50110105,삼도일동
5,50110106,삼도이동
6,50110107,건입동
7,50110108,용담일동
8,50110109,용담이동
9,50110110,용담삼동


In [180]:
EMD=pd.merge(EMD_mean_price2,EMD_with_duplicates_name2, on='코드')

In [181]:
EMD

,코드,거래금액,읍면동
0,50110101,136302.0,일도일동
1,50110102,32269.0,일도이동
2,50110103,59242.0,이도일동
3,50110104,71972.0,이도이동
4,50110105,25025.0,삼도일동
5,50110106,30978.0,삼도이동
6,50110107,29465.0,건입동
7,50110108,27860.0,용담일동
8,50110109,18000.0,용담이동
9,50110110,34333.0,용담삼동


In [182]:
EMD.to_csv("EMD.csv", index=False, encoding='utf-8-sig')

# 제주 "리" 평균 거래금액

In [76]:
jeju_LI=jeju_df[jeju_df['코드']%100!=0]
jeju_LI

,시군구,전용/연면적,거래금액,평당 거래금액,코드,읍면동,리
2,제주특별자치도 서귀포시 남원읍 위미리,395.01,210000,531,5013025327,남원읍,위미리
3,제주특별자치도 서귀포시 남원읍 위미리,40.80,13500,330,5013025327,남원읍,위미리
4,제주특별자치도 서귀포시 남원읍 위미리,40.80,15500,379,5013025327,남원읍,위미리
5,제주특별자치도 서귀포시 남원읍 위미리,62.40,13000,208,5013025327,남원읍,위미리
6,제주특별자치도 서귀포시 남원읍 위미리,62.40,14200,227,5013025327,남원읍,위미리
...,...,...,...,...,...,...,...
428,제주특별자치도 제주시 한림읍 대림리,2943.87,400000,135,5011025023,한림읍,대림리
429,제주특별자치도 제주시 한림읍 옹포리,110.33,27000,244,5011025028,한림읍,옹포리
430,제주특별자치도 제주시 한림읍 옹포리,15.91,5500,345,5011025028,한림읍,옹포리
431,제주특별자치도 제주시 한림읍 월림리,72.44,28000,386,5011025034,한림읍,월림리


In [132]:
LI_mean_price=jeju_LI.groupby(['코드'])['거래금액'].mean().round(0)
LI=pd.DataFrame(LI_mean_price)

In [133]:
LI.reset_index(inplace=True)

In [103]:
jeju_li_only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89 entries, 0 to 446
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   코드      89 non-null     int64 
 1   리       48 non-null     object
dtypes: int64(1), object(1)
memory usage: 2.1+ KB


In [136]:
jeju_li_only.reset_index(inplace=True)

In [121]:
jeju_li_only=jeju_li_only[jeju_li_only['코드']%100!=0]

In [138]:
jeju_li_only.drop(columns=['index'], inplace=True)

/Users/jinwon-kim/Developer/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [141]:
LI=pd.merge(LI, jeju_li_only, on='코드')

#읍면동은 geojson에서 코드 맨뒤 00 생략 그리고 코드는 str타입임..  
LI['코드']=LI['코드'].astype(str)

In [143]:
LI

,코드,거래금액,리
0,5011025023,400000.0,대림리
1,5011025024,29800.0,한림리
2,5011025025,20500.0,금악리
3,5011025028,16250.0,옹포리
4,5011025033,19000.0,금능리
5,5011025034,28000.0,월림리
6,5011025321,70000.0,광령리
7,5011025326,148990.0,신엄리
8,5011025331,17000.0,상가리
9,5011025333,30000.0,고내리


In [144]:
LI.to_csv("LI.csv", index=False, encoding='utf-8-sig')